In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
#import zip codes with respective counties
zips = pd.read_csv('california zip code counties.csv')
zips.head()

,Zip Code,City,County,Zip Code Map
0,90001,Los Angeles,Los Angeles,View Map
1,90002,Los Angeles,Los Angeles,View Map
2,90003,Los Angeles,Los Angeles,View Map
3,90004,Los Angeles,Los Angeles,View Map
4,90005,Los Angeles,Los Angeles,View Map


In [3]:
#drop column and examine data
zips.drop(['Zip Code Map'], axis=1, inplace=True)
zips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2657 entries, 0 to 2656
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Zip Code  2657 non-null   int64 
 1   City      2657 non-null   object
 2   County    2657 non-null   object
dtypes: int64(1), object(2)
memory usage: 41.6+ KB


In [4]:
#filter data to only the six counties in Southern California that we are examining
zips = zips[(zips['County']=='Los Angeles') | (zips['County']=='Orange') | (zips['County']=='Riverside') | 
            (zips['County']=='Ventura') | (zips['County']=='San Bernadino') | (zips['County']=='San Diego')]
zips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1012 entries, 0 to 1358
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Zip Code  1012 non-null   int64 
 1   City      1012 non-null   object
 2   County    1012 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.7+ KB


In [5]:
#import income data
income = pd.read_csv('export-income.xls')
income.head()

,Selected Geographies,Family income (2018)
0,Los Angeles County (North/Unincorporated)--Cas...,53409.63434
1,Los Angeles County (Northwest)--Santa Clarita ...,75121.93139
2,Los Angeles County (North Central)--Lancaster ...,22064.93431
3,Los Angeles County (North Central)--Palmdale C...,50508.47866
4,Los Angeles County (North)--LA City (Northwest...,61025.86957


In [6]:
#rename selected geographies to make split easier
income['Geographies'] = income['Selected Geographies']
income.head()

,Selected Geographies,Family income (2018),Geographies
0,Los Angeles County (North/Unincorporated)--Cas...,53409.63434,Los Angeles County (North/Unincorporated)--Cas...
1,Los Angeles County (Northwest)--Santa Clarita ...,75121.93139,Los Angeles County (Northwest)--Santa Clarita ...
2,Los Angeles County (North Central)--Lancaster ...,22064.93431,Los Angeles County (North Central)--Lancaster ...
3,Los Angeles County (North Central)--Palmdale C...,50508.47866,Los Angeles County (North Central)--Palmdale C...
4,Los Angeles County (North)--LA City (Northwest...,61025.86957,Los Angeles County (North)--LA City (Northwest...


In [7]:
#split geographies column into county and city
income[['County', 'City']] = income.Geographies.str.split(r"--", n=1, expand=True)  
income.head()

,Selected Geographies,Family income (2018),Geographies,County,City
0,Los Angeles County (North/Unincorporated)--Cas...,53409.63434,Los Angeles County (North/Unincorporated)--Cas...,Los Angeles County (North/Unincorporated),Castaic
1,Los Angeles County (Northwest)--Santa Clarita ...,75121.93139,Los Angeles County (Northwest)--Santa Clarita ...,Los Angeles County (Northwest),Santa Clarita City
2,Los Angeles County (North Central)--Lancaster ...,22064.93431,Los Angeles County (North Central)--Lancaster ...,Los Angeles County (North Central),Lancaster City
3,Los Angeles County (North Central)--Palmdale C...,50508.47866,Los Angeles County (North Central)--Palmdale C...,Los Angeles County (North Central),Palmdale City
4,Los Angeles County (North)--LA City (Northwest...,61025.86957,Los Angeles County (North)--LA City (Northwest...,Los Angeles County (North),LA City (Northwest/Chatsworth & Porter Ranch)


In [ ]:
#drop columns with duplicate data
income.drop(['Selected Geographies', 'Geographies'], axis=1, inplace=True)

In [10]:
income

,Family income (2018),County,City
0,53409.63434,Los Angeles County (North/Unincorporated),Castaic
1,75121.93139,Los Angeles County (Northwest),Santa Clarita City
2,22064.93431,Los Angeles County (North Central),Lancaster City
3,50508.47866,Los Angeles County (North Central),Palmdale City
4,61025.86957,Los Angeles County (North),LA City (Northwest/Chatsworth & Porter Ranch)
...,...,...,...
140,99256.87601,Ventura County (Southeast),Thousand Oaks City
141,41539.15282,Ventura County (Southwest),Oxnard & Port Hueneme Cities
142,43424.07678,Ventura County (Southwest),San Buenaventura (Ventura) City
143,44334.57198,Ventura County (North),"Santa Paula, Fillmore & Ojai Cities"


In [11]:
#add income column to zips df
zips['Income'] = ""
zips.tail(10)

,Zip Code,City,County,Income
1339,93550,Palmdale,Los Angeles,
1340,93551,Palmdale,Los Angeles,
1341,93552,Palmdale,Los Angeles,
1342,93553,Pearblossom,Los Angeles,
1350,93563,Valyermo,Los Angeles,
1352,93584,Lancaster,Los Angeles,
1353,93586,Lancaster,Los Angeles,
1354,93590,Palmdale,Los Angeles,
1355,93591,Palmdale,Los Angeles,
1358,93599,Palmdale,Los Angeles,


In [12]:
for index_zips, row_zips in zips.iterrows():
    for index_income, row_income in income.iterrows():
        if zips.loc[index_zips, 'City'] in income.loc[index_income, 'City']:
            zips.loc[index_zips, 'Income'] = income.loc[index_income, 'Family income (2018)']
            
zips.tail(10)

,Zip Code,City,County,Income
1339,93550,Palmdale,Los Angeles,50508.5
1340,93551,Palmdale,Los Angeles,50508.5
1341,93552,Palmdale,Los Angeles,50508.5
1342,93553,Pearblossom,Los Angeles,
1350,93563,Valyermo,Los Angeles,
1352,93584,Lancaster,Los Angeles,22064.9
1353,93586,Lancaster,Los Angeles,22064.9
1354,93590,Palmdale,Los Angeles,50508.5
1355,93591,Palmdale,Los Angeles,50508.5
1358,93599,Palmdale,Los Angeles,50508.5


In [13]:
zips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1012 entries, 0 to 1358
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Zip Code  1012 non-null   int64 
 1   City      1012 non-null   object
 2   County    1012 non-null   object
 3   Income    1012 non-null   object
dtypes: int64(1), object(3)
memory usage: 59.7+ KB


In [14]:
zips['Income'] = pd.to_numeric(zips['Income'], errors = 'coerce')
zips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1012 entries, 0 to 1358
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Zip Code  1012 non-null   int64  
 1   City      1012 non-null   object 
 2   County    1012 non-null   object 
 3   Income    855 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 63.6+ KB


In [ ]:
zips.tail(10)

In [15]:
negdata = zips[(zips['Income']<1)]
negdata

,Zip Code,City,County,Income
248,90755,Signal Hill,Los Angeles,-6851.478060
249,90801,Long Beach,Los Angeles,-6851.478060
250,90802,Long Beach,Los Angeles,-6851.478060
251,90803,Long Beach,Los Angeles,-6851.478060
252,90804,Long Beach,Los Angeles,-6851.478060
253,90805,Long Beach,Los Angeles,-6851.478060
254,90806,Long Beach,Los Angeles,-6851.478060
255,90807,Long Beach,Los Angeles,-6851.478060
256,90808,Long Beach,Los Angeles,-6851.478060
257,90809,Long Beach,Los Angeles,-6851.478060


In [ ]:
zips.loc[zips['Income'].isnull(), 'value_is_NaN'] = 'Yes'
zips.loc[zips['Income'].notnull(), 'value_is_NaN'] = 'No'
count_nan = zips.loc[zips['value_is_NaN']=='Yes'].count()
count_nan

In [ ]:
nodata = zips[(zips['value_is_NaN'] == 'Yes')]
nodata

In [16]:
#because there are only 5 cities with negative income values, I exported new data for those areas from the U.S. Census Bureau
#census.gov/quickfacts
census = pd.read_csv('QuickFacts-negzipIncome.csv')

In [17]:
census.drop(range(0,43), axis=0, inplace=True)
census.drop([43], axis=0, inplace=True)
census

,Fact,Fact Note,"Blythe city, California","Value Note for Blythe city, California","Coachella city, California","Value Note for Coachella city, California","Indio city, California","Value Note for Indio city, California","Long Beach city, California","Value Note for Long Beach city, California","Signal Hill city, California","Value Note for Signal Hill city, California"
44,"Median household income (in 2019 dollars), 201...",NaN,"$45,385",NaN,"$34,224",NaN,"$53,669",NaN,"$63,017",NaN,"$75,508",NaN
45,Per capita income in past 12 months (in 2019 d...,NaN,"$17,143",NaN,"$17,442",NaN,"$27,211",NaN,"$32,323",NaN,"$40,286",NaN
46,"Persons in poverty, percent",NaN,25.5%,NaN,21.8%,NaN,16.5%,NaN,16.8%,NaN,16.4%,NaN
47,"Total employer establishments, 2018",NaN,X,NaN,X,NaN,X,NaN,X,NaN,X,NaN
48,"Total employment, 2018",NaN,X,NaN,X,NaN,X,NaN,X,NaN,X,NaN
49,"Total annual payroll, 2018 ($1,000)",NaN,X,NaN,X,NaN,X,NaN,X,NaN,X,NaN
50,"Total employment, percent change, 2017-2018",NaN,X,NaN,X,NaN,X,NaN,X,NaN,X,NaN
51,"Total nonemployer establishments, 2018",NaN,X,NaN,X,NaN,X,NaN,X,NaN,X,NaN
52,"All firms, 2012",NaN,901,NaN,"2,998",NaN,"6,222",NaN,"43,731",NaN,"2,024",NaN
53,"Men-owned firms, 2012",NaN,292,NaN,"1,484",NaN,"2,893",NaN,"21,681",NaN,"1,114",NaN


In [18]:
census.drop(range(45,82), axis=0, inplace=True)
census

,Fact,Fact Note,"Blythe city, California","Value Note for Blythe city, California","Coachella city, California","Value Note for Coachella city, California","Indio city, California","Value Note for Indio city, California","Long Beach city, California","Value Note for Long Beach city, California","Signal Hill city, California","Value Note for Signal Hill city, California"
44,"Median household income (in 2019 dollars), 201...",NaN,"$45,385",NaN,"$34,224",NaN,"$53,669",NaN,"$63,017",NaN,"$75,508",NaN
82,Z,Value greater than zero but less than half uni...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
census.drop(['Fact','Fact Note', 'Value Note for Blythe city, California', 'Value Note for Coachella city, California', 
             'Value Note for Indio city, California', 'Value Note for Long Beach city, California', 
             'Value Note for Signal Hill city, California'], axis=1, inplace=True)
census.drop([82], axis=0, inplace=True)
census

,"Blythe city, California","Coachella city, California","Indio city, California","Long Beach city, California","Signal Hill city, California"
44,"$45,385","$34,224","$53,669","$63,017","$75,508"


In [21]:
censust = census.transpose()
censust

,44
"Blythe city, California","$45,385"
"Coachella city, California","$34,224"
"Indio city, California","$53,669"
"Long Beach city, California","$63,017"
"Signal Hill city, California","$75,508"


In [22]:
censust[44] =censust[44].str.replace(',','')
censust[44] =censust[44].str.replace('$','')
censust[44] = pd.to_numeric(censust[44])
censust

,44
"Blythe city, California",45385
"Coachella city, California",34224
"Indio city, California",53669
"Long Beach city, California",63017
"Signal Hill city, California",75508


In [23]:
for index_zips, row_zips in zips.iterrows():
    for index_censust, row_censust in censust.iterrows():
        if zips.loc[index_zips, 'City'] in index_censust:
            zips.loc[index_zips, 'Income'] = censust.loc[index_censust, 44]


In [ ]:
print(zips[np.where(zips['City'] == 'Long Beach')])

In [24]:
zips[248:278]

,Zip Code,City,County,Income
248,90755,Signal Hill,Los Angeles,75508.00000
249,90801,Long Beach,Los Angeles,63017.00000
250,90802,Long Beach,Los Angeles,63017.00000
251,90803,Long Beach,Los Angeles,63017.00000
252,90804,Long Beach,Los Angeles,63017.00000
253,90805,Long Beach,Los Angeles,63017.00000
254,90806,Long Beach,Los Angeles,63017.00000
255,90807,Long Beach,Los Angeles,63017.00000
256,90808,Long Beach,Los Angeles,63017.00000
257,90809,Long Beach,Los Angeles,63017.00000


In [25]:
zips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1012 entries, 0 to 1358
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Zip Code  1012 non-null   int64  
 1   City      1012 non-null   object 
 2   County    1012 non-null   object 
 3   Income    855 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 63.6+ KB


In [26]:
zips.to_csv('incomeclean.csv')